In [1]:
import psycopg2 as pg2
import pandas as pd

con = pg2.connect(host='this_postgres',
                  user='postgres',
                  password='password',
                  database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

Check running processes

In [16]:
sql = '''
SELECT pid , query, *
FROM pg_stat_activity pgsa
WHERE pid <> pg_backend_pid()
'''

In [17]:
df = select(sql)

In [18]:
df

,pid,query,datid,datname,pid,usesysid,usename,application_name,client_addr,client_hostname,...,xact_start,query_start,state_change,wait_event_type,wait_event,state,backend_xid,backend_xmin,query,backend_type
0,60,,NaN,None,60,NaN,None,,None,None,...,None,NaT,NaT,Activity,AutoVacuumMain,None,None,None,,autovacuum launcher
1,61,/*pga4dash*/\nSELECT 'session_stats' AS chart_...,13075.0,postgres,61,10.0,postgres,pgAdmin 4 - DB:postgres,172.23.0.1,None,...,None,2021-01-19 08:52:54.898455+00:00,2021-01-19 08:52:54.916552+00:00,Client,ClientRead,idle,None,None,/*pga4dash*/\nSELECT 'session_stats' AS chart_...,client backend
2,23,,NaN,None,23,NaN,None,,None,None,...,None,NaT,NaT,Activity,BgWriterHibernate,None,None,None,,background writer
3,22,,NaN,None,22,NaN,None,,None,None,...,None,NaT,NaT,Activity,CheckpointerMain,None,None,None,,checkpointer
4,24,,NaN,None,24,NaN,None,,None,None,...,None,NaT,NaT,Activity,WalWriterMain,None,None,None,,walwriter


Kill all

In [14]:
sql = '''
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    -- don't kill my own connection!
    pid <> pg_backend_pid()

'''

In [15]:
cur.execute(sql)